# Creating Gait Analysis Dataset

## In this code, our aim is to read data from excel files, run Segmentation algorithm on it and then extract features for each gait cycle and save them as datasets

In [16]:
#%% Importing
from  funcs import *
import numpy as np
import matplotlib.pylab as plt
from scipy import signal
import seaborn as sns
import pandas as pd

from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector
from keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.models import Model

In [3]:
#%% Initiation
main_dir = r'D:\University\Khaje\Semester 1\Neuromuscular\Assymetry Project\Datas'
file_names = os.listdir(main_dir)

In [4]:
file_names

['A', 'B', 'F', 'M']

# Reading datas from files into one DataFrame

#### This code loops over 4 patients and 8 tests for each of them and read the data of corresponding excel file into the data frame named df. then the Segmentation Algorithm runs over the excel file and create two objects named "cycles" and "cycles_r" for right and left gait cycles. 
#### for our purpose, we need to choose only one sensor and one body part. for now, we have been working on sagittal sensor of ankles.
#### afterward, we  run a feature extraction algorithm named TSFEL on each time series and read it into a dataframe named "features". two labeles of 'Patient' and 'Test' is appended to features and finally this dataframe is concatenated to main dataframes 'Right_all_features' and  'Left_all_features'

In [271]:
import os
import pandas as pd
from tsfel import time_series_features_extractor

# The main directory where the patient folders are located
main_dir = r'D:\University\Khaje\Semester 1\Neuromuscular\Assymetry Project\Datas'

# Initialize an empty list to store all dataframes
all_data = []

#Right_Features = np.zeros(100)
Left_all_features = pd.DataFrame()
Right_all_features = pd.DataFrame()
# Initialize a counter
counter = 0

# Loop over each patient folder
for patient in os.listdir(main_dir):
    patient_dir = os.path.join(main_dir, patient)
    
    # Check if it's a directory
    if os.path.isdir(patient_dir):
        
        # Loop over each test folder in the patient folder
        for test in os.listdir(patient_dir):
            test_dir = os.path.join(patient_dir, test)
            
            # Check if it's a directory
            if os.path.isdir(test_dir):
                
                # Loop over each file in the test folder
                for file in os.listdir(test_dir):
                    file_path = os.path.join(test_dir, file)
                    
                    # Check if it's a file
                    if os.path.isfile(file_path):
                        
                        # Read the Excel file into a DataFrame
                        df = pd.read_excel(file_path, header=1, engine='openpyxl')  # for .xlsx files
                        
                        '''
                        Segmentation: Segment dataframe into segments based on right and left hip signals.
                        it then generates two objects cycles and cycles_r.
                        '''
                        
                        # Creating gaitData obejct out of the main dataframe
                        test_df = gaitData(df)
                        test_df.detrend()
                        # Extracting limits
                        cycle_lim = lim_ex(test_df)
                        cycle_lim_r = lim_ex_r(test_df)
                        # Segmentation
                        cycles = Cycles(test_df, cycle_lim)
                        cycles_r = Cycles(test_df, cycle_lim_r)
                        
                        '''
                        Choose Sensor and Signal (Sagittal,frontal or Horizontal) for next classification steps.
                        this is stored in Right_Signal and Left_Signal variables.
                        Also we need a for loop to loop over each segmet/gait cycle and extract features for it
                        
                        Current Signal: Ankle_Saggital
                        '''
                        cycle_counter = 0
                        for cycle_counter in range(0,len(cycles_r.right_ankle.sagittal)-1):
                            
                            Right_Signal = cycles_r.right_ankle.sagittal.iloc[cycle_counter]
                            
                            
                            '''
                            Feature Extraction. use each signal to create featuresfrom it. it is done in two different parts 
                            for right and left signals.
                            here we are using TSFEL to extract features. the lengh of window is set to the 
                            length of the signal so that we have only one set of features for each gait cycle
                            frequency is equal to 166 because every 0.006 seconds, we take a sample
                            ''' 
                            # Right Feature Extraction
                            
                            data = pd.DataFrame(Right_Signal)
                            if len(data) > 15:
                                # Load your time series data into a pandas DataFrame
                                cfg_file = tsfel.get_features_by_domain()  

                                # Extract features using TSFEL
                                features = time_series_features_extractor(cfg_file,data, fs = 166, window_size=len(data) )  # fs is the sampling frequency
                                # Add patient and test labels to features for further analysis
                            
                                features['Patient'] = patient
                                features['Test'] = test
                                #concat features into a single dataframe
                                Right_all_features =  pd.concat([Right_all_features,features], ignore_index=True)
                                
                            print(f'Processed {counter} files ,{cycle_counter}/{len(cycles_r.right_ankle.sagittal)}')    
                        cycle_counter = 0
                        for cycle_counter in range(0,len(cycles.left_ankle.sagittal)-1):
                            Left_Signal = cycles.left_ankle.sagittal.iloc[cycle_counter]
                        
                            
                            #Left Feature Extraction
                            
                            data = pd.DataFrame(Left_Signal)
                            if len(data) > 15:
                                # Load your time series data into a pandas DataFrame
                                cfg_file = tsfel.get_features_by_domain()  

                                # Extract features using TSFEL
                                features = time_series_features_extractor(cfg_file,data, fs = 166, window_size=len(data) )  # fs is the sampling frequency
                                # Add patient and test labels to features for further analysis
                            
                                features['Patient'] = patient
                                features['Test'] = test
                                #concat features into a single dataframe
                                Left_all_features = pd.concat([Left_all_features,features], ignore_index=True)
                            print(f'Processed {counter} files ,{cycle_counter}/{len(cycles.left_ankle.sagittal)}')
                            
                            
                            
                            
                            ###############
                        # Increment the counter and print it
                        counter += 1


# Concatenate all the dataframes in the list into a single dataframe
#all_data = pd.concat(all_data, ignore_index=True)
#Right_all_features = pd.concat(Right_all_features, ignore_index=True)
#Left_all_features = pd.concat(Left_all_features, ignore_index=True)

# Now all_data contains all the data from the Excel files, with additional columns for the patient and test names



*** Feature extraction started ***



*** Feature extraction finished ***
Processed 0 files ,0/41
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 0 files ,1/41
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 0 files ,2/41
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 0 files ,3/41
*** Feature extraction started ***


...


*** Feature extraction finished ***
Processed 32 files ,86/92
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 32 files ,87/92
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 32 files ,88/92
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 32 files ,89/92
*** Feature extraction started ***



*** Feature extraction finished ***
Processed 32 files ,90/92


## Saving resulting dataframes of features

In [279]:
Left_all_features.to_excel('Left_Ankle_Sagittal_Features.xlsx', index=False)

In [280]:
Right_all_features.to_excel('Right_Ankle_Sagittal_Features.xlsx', index=False)

In [273]:
Left_all_features

,sagittal_Absolute energy,sagittal_Area under the curve,sagittal_Autocorrelation,sagittal_Average power,sagittal_Centroid,sagittal_ECDF Percentile Count_0,sagittal_ECDF Percentile Count_1,sagittal_ECDF Percentile_0,sagittal_ECDF Percentile_1,sagittal_ECDF_0,...,sagittal_FFT mean coefficient_240,sagittal_FFT mean coefficient_241,sagittal_FFT mean coefficient_242,sagittal_FFT mean coefficient_243,sagittal_FFT mean coefficient_244,sagittal_FFT mean coefficient_245,sagittal_FFT mean coefficient_246,sagittal_FFT mean coefficient_247,sagittal_FFT mean coefficient_248,sagittal_FFT mean coefficient_249
0,571.050330,0.628224,571.050330,4739.717741,0.059985,4.0,16.0,-5.241236,-5.197069,0.047619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1539.171338,1.799908,1539.171338,4330.549865,0.172917,12.0,48.0,-5.216367,-4.943692,0.016667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1466.931429,1.801580,1466.931429,3927.590601,0.189294,12.0,50.0,-4.899513,-4.737341,0.015873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7358.493861,7.332893,7358.493861,4662.251836,0.865192,52.0,210.0,-4.839752,6.270216,0.003802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13695.165707,7.558404,13695.165707,14858.807237,0.354518,30.0,123.0,-1.622858,8.454327,0.006494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1805,31315.213426,8.330416,31315.213426,38793.473349,0.327828,27.0,108.0,-21.886719,0.343238,0.007407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806,2722.257257,2.669468,2722.257257,2860.093068,0.186463,31.0,127.0,-0.929169,2.691893,0.006289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1807,26728.594135,7.364550,26728.594135,37922.620738,0.336259,23.0,94.0,-23.119412,1.553692,0.008475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1808,3846.697915,1.443631,3846.697915,22019.029443,0.150680,6.0,24.0,2.332091,14.336422,0.033333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
Right_all_features

,sagittal_Absolute energy,sagittal_Area under the curve,sagittal_Autocorrelation,sagittal_Average power,sagittal_Centroid,sagittal_ECDF Percentile Count_0,sagittal_ECDF Percentile Count_1,sagittal_ECDF Percentile_0,sagittal_ECDF Percentile_1,sagittal_ECDF_0,...,sagittal_FFT mean coefficient_246,sagittal_FFT mean coefficient_247,sagittal_FFT mean coefficient_248,sagittal_FFT mean coefficient_249,sagittal_FFT mean coefficient_250,sagittal_FFT mean coefficient_251,sagittal_FFT mean coefficient_252,sagittal_FFT mean coefficient_253,sagittal_FFT mean coefficient_254,sagittal_FFT mean coefficient_255
0,2442.535344,2.840512,2442.535344,4407.183338,0.280271,18.0,74.0,5.053906,5.187684,0.010753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2287.799788,2.703107,2287.799788,4267.132189,0.266106,18.0,72.0,4.995754,5.095390,0.011111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2360.340011,2.757560,2360.340011,4353.516021,0.278526,18.0,72.0,4.998535,5.043029,0.010989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17745.556555,8.543786,17745.556555,14511.144769,0.898400,40.0,163.0,-6.567495,5.621806,0.004902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16563.661340,6.205173,16563.661340,18089.261726,0.673809,30.0,122.0,-4.469373,5.586578,0.006536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,22434.455347,8.448096,22434.455347,16052.239602,1.089339,46.0,186.0,-2.373927,9.062551,0.004292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1588,3686.582303,2.766653,3686.582303,8999.597975,0.236706,13.0,55.0,-7.656718,-4.316572,0.014493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1589,21164.275147,7.982204,21164.275147,15969.407611,1.081892,44.0,176.0,-2.118617,11.099019,0.004525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590,963.198016,1.266077,963.198016,2220.706536,0.291905,14.0,58.0,-3.214288,2.707258,0.013699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Thank you